# Lecture 8 - PyTorch

This will be the final lecture, today we will first have a brief introduction of deep learning, then we will look at some basics of using PyTorch to implement some simple models in deep learning.



## Deep Learning Libraries

There are many deep learning libraries available, the most common ones for python are

- TensorFlow, Keras
- PyTorch

Working with tensorflow requires going into lot of details of the contruction of the computation graph, whereas Keras is a higher level interface for tensorflow. Tensorflow is very popular in the industry and good for production code.

PyTorch can be used as low level interface, but is much more user-friendly than tensorflow, but it also has a higher level interface. Pytorch is more popular in the research community and the 1.0 version was only recently released.


## Main features that any deep learning library should provide

No matter what library or language you use, the main features provided by a deep learning library are 
1. Use the GPU to speed up computation 
2. Ability to do automatic differentiation
3. Useful library functions for common architectures and optimization algorithms

### PyTorch
We will look at all of the above in pytorch.
The best way to think about pytorch is that its numpy + GPU + autograd.

You can install it with

```conda install pytorch```

In [ ]:
import torch

The equivalent object to numpy arrays in pytorch are called tensors, but they are just multidimensional arrays.

In [ ]:
torch.tensor([2,3,4,5])

In [ ]:
torch.zeros((5,5))

In [ ]:
x = torch.ones((5,5))
x

In [ ]:
2*x + 5

In [ ]:
torch.rand(5,5)

In [ ]:
x = torch.rand(25)
x

In [ ]:
x=x.reshape(-1,5)
x

In [ ]:
x.shape

In [ ]:
print(torch.arange(10))
print(torch.eye(5))
print(torch.linspace(0,1,10))

Some functions are a bit different

In [ ]:
A = torch.rand(5,5)
x = torch.ones(5,1)
A.mm(x)

In [ ]:
import numpy as np
A = np.random.rand(5,5)
x = np.ones((5,1))
A.dot(x)

You can convert tensors to a numpy array that shares its memory with the pytorch tensor

In [ ]:
x = torch.ones(5,5)
x

In [ ]:
xn = x.numpy()
xn

In [ ]:
xn[4,2]=10
xn

In [ ]:
x

### Using the GPU

The GPU (Graphical Processing Unit) is a separate processing unit that is specialized to handle bulk computations required for rendering high quality graphics. It mainly consists of a large number of processor cores that are individually very slow, but because of their sheer number (around 2000) they can churn through computations very quickly. 

In [ ]:
import torch
torch.cuda.is_available()

Installing the GPU drivers and the CUDA toolkit can be quite messy, so if you just want to experiment with GPUs and deep learning libraries, you can use [Google colaboratory](https://colab.research.google.com/)

In [ ]:
gpu = torch.device("cuda")
cpu = torch.device("cpu")

In [ ]:
A = torch.rand(100,100)
B = torch.rand(100,100)

In [ ]:
A.mm(B)

In [ ]:
A_gpu = A.to(gpu)
B_gpu = B.to(gpu)

In [ ]:
A_gpu.mm(B_gpu)

In [ ]:
A.mm(B_gpu)

In [ ]:
C_gpu = A_gpu.mm(B_gpu)
C = C_gpu.to(cpu)
C

### GPU - CPU memory transfer

In [ ]:
big_mat = torch.rand(20000,20000);

In [ ]:
big_mat_gpu = big_mat.to(gpu)

In [ ]:
big_mat= big_mat_gpu.to(cpu)

In [ ]:
del big_mat_gpu
torch.cuda.empty_cache()

In [ ]:
del big_mat

## Speedup from GPU

In [ ]:
%%timeit
A = torch.rand(3000,3000)
B = torch.rand(3000,3000)
C = torch.zeros(3000,3000)
C.copy_(B)
for i in range(5):
    C=torch.mm(A,C)

In [ ]:
%%timeit
A = torch.rand(3000,3000, device = gpu)
B = torch.rand(3000,3000, device = gpu)
C = torch.zeros(3000,3000, device = gpu)
C.copy_(B)
for i in range(5):
    C=torch.mm(A,C)

## Automatic Differentiation

PyTorch uses dynamic computation graphs to compute the gradients of the parameters.

In [ ]:
x = torch.tensor([2.0])
m = torch.tensor([5.0], requires_grad = True)
c = torch.tensor([2.0], requires_grad = True)

In [ ]:
y = m*x + c
y

Define an error for your function

In [ ]:
loss = torch.norm( y - 13)

In [ ]:
m.grad

Calling `x.backward()` on any tensor forces pytorch to compute all the gradients of the tensors used to compute `x` which had the `requires_grad` flag set to `True`. The computed gradient will be stored in the `.grad` property of the tensors

In [ ]:
loss.backward()

In [ ]:
m.grad

In [ ]:
c.grad

In [ ]:
with torch.no_grad():
    m -= 0.01 * m.grad
    c -= 0.01 * c.grad

In [ ]:
m,c

In [ ]:
m.grad, c.grad

In [ ]:
m.grad.zero_()
c.grad.zero_()

m.grad, c.grad

In [ ]:
y = m*x + c

In [ ]:
y

In [ ]:
loss = torch.norm( y - 13)

In [ ]:
loss.backward()
m.grad, c.grad

### Making it more compact

In [ ]:
def model_fn(x,m,c):
    return m*x + c

In [ ]:
def loss_fn(y,yt):
    return torch.norm(y-yt)

In [ ]:
m = torch.tensor([5.0], requires_grad = True)
c = torch.tensor([2.0], requires_grad = True)

In [ ]:
x = torch.tensor([2.0])
yt = torch.tensor([13.0])

In [ ]:
y = model_fn(x,m,c)
loss = loss_fn(y,yt)
loss.backward()
with torch.no_grad():
    m -= 0.05 * m.grad
    c -= 0.05 * c.grad
m.grad.zero_()
c.grad.zero_()

print( f" m = {m}\n c = {c}\n y = {y}\n loss = {loss}")

### Slightly more complicated problem

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def model_fn(x,m,c):
    return m@x + c

In [ ]:
def loss_fn(y,yt):
    return torch.norm(y-yt)

In [ ]:
m = torch.rand((5,5), requires_grad = True)
c = torch.ones((5,1), requires_grad = True)

In [ ]:
x = torch.randn(5,100)
yt = torch.randn(1,100)
losses = []

In [ ]:
y = model_fn(x,m,c)
loss = loss_fn(y,yt)
loss.backward()
with torch.no_grad():
    m -= 0.05 * m.grad
    c -= 0.05 * c.grad
m.grad.zero_()
c.grad.zero_()

losses+=[loss.item()]
print( f"loss = {loss}")
plt.plot(losses);

## Using Library functions

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(5, 5),
    torch.nn.ReLU(),
    torch.nn.Linear(5, 5),
)

In [ ]:
list(model.parameters())

In [ ]:
loss_fn = torch.nn.MSELoss(reduction='sum')

In [ ]:
x = torch.randn(100,5)
yt = torch.randn(100,1)
losses = []

In [ ]:
y = model(x)
loss = loss_fn(y,yt)
loss.backward()
with torch.no_grad():
    for param in model.parameters():
        param -= 0.01 * param.grad
        
model.zero_grad()

losses+=[loss.item()]
print( f"loss = {loss}")
plt.plot(losses);

Using the optim package

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)

In [ ]:
y = model(x)
loss = loss_fn(y,yt)
loss.backward()

optimizer.step()
optimizer.zero_grad()

losses+=[loss.item()]
print( f"loss = {loss}")
plt.plot(losses);

## MNIST Example

In [ ]:
from torchvision.datasets import MNIST

In [ ]:
data = MNIST(".",download=True)

In [ ]:
img,y = data[2]

In [ ]:
img

In [ ]:
y

In [ ]:
data.train_data[2].shape

In [ ]:
data.train_labels[2]

### MNIST Training

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(784, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 10),
)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
sample = np.random.choice(range(len(data.train_data)),1000)
x = data.train_data[sample].reshape(1000,-1).float()/255
yt = data.train_labels[sample]

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
losses = []

In [ ]:
for i in range(100):
    y = model(x)
    loss = loss_fn(y,yt)
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    losses+=[loss.item()]
    print( f"loss = {loss}")
plt.plot(losses);

In [ ]:
x_test = data.train_data[-1000:].reshape(1000,-1).float()/255
y_test = data.train_labels[-1000:]

In [ ]:
with torch.no_grad():
    y_pred = model(x_test)

In [ ]:
print("Accuracy = ", (y_pred.argmax(dim=1) == y_test).sum().float().item()/1000.0)

## Course Conclusion

By now you should have a sufficient introduction to the various ways one can use python for scientific computing. The best way to learn more is to start using python for whatever project you are working on. Only practice will make you comfortable with using python.

### Reminders
    - Assignment 2 is due Feb 17, 5PM
    - There will be office hours after class now
    - There will be office hours next thursday as well from 10AM to 12PM
    - You can also email me and schedule office hours by appointment, if you cannot make it during the above times

In [ ]:
#